<a href="https://colab.research.google.com/github/Manisha2297/RestroRecommender/blob/main/NLTKmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
import string
from scipy import sparse

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#mount your own drive to avoid downloading the data multiple time 

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
df_review = pd.read_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/review_filtered.csv')
df_review.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# df_review = pd.read_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/yelp_academic_dataset_review.csv')
# df_review.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
restaurant_df = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/restuarant_df')

In [7]:
restaurant_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,404 E Green St,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.1104,-88.2331,The Empanadas House,61820,5,4.5,IL
1,2831 Parmenter St,"{'BikeParking': 'True', 'Caters': 'True', 'Whe...",M_guz7Dj7hX0evS672wIwA,"Desserts, Food, Ice Cream & Frozen Yogurt",Middleton,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",1,43.1053,-89.5101,Chocolate Shoppe Ice Cream,53562,6,3.5,WI
2,NaN,{'BusinessAcceptsCreditCards': 'True'},mKTq1T_IAplDpHUcMzOXkw,"Hotels & Travel, Transportation, Taxis, Beer, ...",Gastonia,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,35.2528,-81.1527,Tipsy Ryde,28054,3,3.5,NC
3,"15480 Bayview Avenue, unit D0110","{'RestaurantsTableService': 'False', 'Restaura...",eBEfgOPG7pvFhb2wcG9I7w,"Restaurants, Cheesesteaks, Poutineries",Aurora,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,44.011,-79.4487,Philthy Phillys,L4G 7J1,4,4.5,ON
4,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,NaN,1,43.8205,-79.3985,Banzai Sushi,L3T 5W4,7,4.5,ON


In [6]:
# NC
business_id_list = list(restaurant_df[restaurant_df['state']=='NV']['business_id'].values)

In [ ]:
# df_review_filtered = df_review[df_review['business_id'].isin(business_id_list)]

In [ ]:
# pd.DataFrame(df_review_filtered.user_id.unique(), columns=['user_id']).to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/unique_users_df')

In [ ]:
# df_review_filtered.to_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/review_filtered_ON.csv')

In [9]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,HQl28KMwrEKHqhFrrDqVNQ,0,2015-12-05 03:18:11,0.0,LG2ZaYiOgpr2DK_90pYjNw,5.0,I love Deagan's. I do. I really do. The atmosp...,1.0,V34qejxNsCbcgD8C0HVk-Q
1,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0.0,6TdNDKywdbjoTkizeMce8A,4.0,"Oh happy day, finally have a Canes near my cas...",0.0,UgMW8bLE0QMJDCkQ1Ax5Mg
2,nlxHRv1zXGT0c0K51q3jDg,0,2013-05-07 07:25:25,0.0,L2O_INwlrRuoX05KSjc4eg,5.0,This is definitely my favorite fast food sub s...,2.0,5vD2kmE25YBrbayKhykNxQ
3,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0.0,ZayJ1zWyWgY9S_TRLT_y9Q,5.0,"Really good place with simple decor, amazing f...",1.0,aq_ZxGHiri48TUXJlpRkCQ
4,e_BiI4ej1CW1F0EyVLr-FQ,0,2015-02-16 06:48:47,0.0,JA-xnyHytKiOIHl_ztnK9Q,5.0,Most delicious authentic Italian I've had in t...,0.0,P6apihD4ASf1vpPxHODxAQ


In [7]:
#Select only stars and text
yelp_data = df_review[['business_id', 'user_id', 'stars', 'text']]

In [8]:
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [9]:
yelp_data['text'].fillna("", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they have any punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

In [ ]:
# size = 0
# processed_text_df = pd.DataFrame()
# for i in range(2000000, len(yelp_data), 10000):
#   processed_text_df = pd.concat([processed_text_df, yelp_data['text'][i:i+10000].apply(text_process)])
#   size+=10000

#   if len(processed_text_df) >= size:
#     if len(processed_text_df)%100000==0:
#       print(len(processed_text_df))
#     processed_text_df.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_2')
#     del processed_text_df
#     processed_text_df = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_2')
  
#   else:
#     print(size)
#     break


In [ ]:
len(yelp_data)

4607291

In [ ]:
# processed_text_df_1 = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_1')

In [ ]:
# processed_text_df_final = pd.concat([processed_text_df_1, processed_text_df])

In [ ]:
# processed_text_df_final.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df')

In [10]:
processed_text_df_final = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df')

In [11]:
yelp_data['text'] = processed_text_df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
yelp_data = yelp_data[yelp_data['business_id'].isin(business_id_list)]

In [13]:
yelp_data.shape

(1518966, 4)

In [14]:
yelp_data.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/yelp_data_df_NV')

In [ ]:
# del processed_text_df_1
del df_review

In [4]:
yelp_data = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/yelp_data_df_NV')

In [15]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [16]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])

In [17]:
sparse.save_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/userid_vectors_NV.npz", userid_vectors)

In [6]:
userid_vectors = sparse.load_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/userid_vectors.npz")

In [18]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])

In [19]:
sparse.save_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/businessid_vectors_NV.npz", businessid_vectors)

In [7]:
businessid_vectors = sparse.load_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/businessid_vectors.npz")

In [5]:
from tqdm import tqdm

chunk_size = 50000
chunks = [x for x in range(0, yelp_data.shape[0], chunk_size)]

for i in range(0, len(chunks) - 1):
    print(chunks[i], chunks[i + 1] - 1)

0 49999
50000 99999
100000 149999
150000 199999
200000 249999
250000 299999
300000 349999
350000 399999
400000 449999
450000 499999
500000 549999


In [ ]:
userid_rating_matrix = pd.DataFrame()
user_col = 'user_id'
item_col = 'business_id'
rating_col = 'stars'

for i in tqdm(range(0, len(chunks) - 1)):
    chunk_df = yelp_data.iloc[ chunks[i]:chunks[i + 1] - 1]
    interactions = (
    chunk_df.groupby([user_col, item_col])[rating_col]
    .sum()
    .unstack()
    .reset_index()
    .fillna(0)
    .set_index(user_col)
    )
    print (interactions.shape)
    userid_rating_matrix = userid_rating_matrix.append(interactions, sort=False) 

  9%|▉         | 1/11 [00:01<00:11,  1.11s/it]

(26390, 1355)
(26537, 1742)


 18%|█▊        | 2/11 [00:03<00:13,  1.49s/it]

(26642, 2441)


 27%|██▋       | 3/11 [00:07<00:17,  2.22s/it]

(26497, 2602)


 36%|███▋      | 4/11 [00:13<00:23,  3.39s/it]

(26968, 2516)


In [20]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])

ValueError: ignored

In [ ]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q


In [ ]:
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

In [ ]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())